# Simple FEM for an elastic plate with a hole

## Create a mesh for the plate with hole

First we need to build the geometry. The shape and mesh is generated with pygmsh, exported to a xdmf file and then imported for use with fenicsx. 

In [79]:
import numpy as np

import pyvista
import pygmsh
import ufl
from dolfinx import io, fem, mesh, plot
from mpi4py import MPI
from petsc4py import PETSc

from global_constants import EPS0, LBD, MU, B0, L, R

# Element size
N = 20
s = L / N

# Build mesh
with pygmsh.occ.Geometry() as geom:
    geom.characteristic_length_max = s
    rectangle = geom.add_rectangle([0.0, 0.0, 0.0], a=L, b=L)
    hole = geom.add_disk([0, 0, 0.0], R)
    geom.boolean_difference(rectangle, hole)
    m = geom.generate_mesh()
    # Reduce cells to triangles
    m.cells = [cells for cells in m.cells if cells.type == "triangle"]
    # Reduce points to 2D
    m.points = m.points[:, :2]
    m.write("mesh.xdmf")


with io.XDMFFile(MPI.COMM_WORLD, "mesh.xdmf", "r") as xdmf:
    domain = xdmf.read_mesh(name="Grid")

# Boundary sets
left_facets = mesh.locate_entities_boundary(
    domain, dim=1, marker=lambda x: np.isclose(x[0], 0.0)
)
right_facets = mesh.locate_entities_boundary(
    domain, dim=1, marker=lambda x: np.isclose(x[0], L)
)
bottom_facets = mesh.locate_entities_boundary(
    domain, dim=1, marker=lambda x: np.isclose(x[1], 0.0)
)


## Create the FEM function space 

In [80]:
V = fem.VectorFunctionSpace(domain, ("Lagrange", 1))


## Problem definition

The weak form of the equation to be solved for the displacement field $\mathbf{u}: \reals^2 \rightarrow \reals^2$ is 

$$ 
    \int_\Omega \sigma \cdot \nabla \mathbf{v} dV - \int_\Omega \mathbf{b} \cdot \mathbf{v} dV = 0
$$
with 
$$
\sigma = 2\mu \epsilon + \lambda \textrm{tr}(\epsilon) \mathbf{I}
$$
and
$$ 
    \varepsilon = \frac{1}{2} \left( \nabla \mathbf{u} + \nabla \mathbf{u}^\top \right).
$$

In [81]:
def eps(v):
    return ufl.sym(ufl.grad(v))


def sigma(v):
    return 2.0 * MU * eps(v) + LBD * ufl.tr(eps(v)) * ufl.Identity(2)


u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
b = fem.Constant(domain, PETSc.ScalarType((0, B0)))
a = ufl.inner(sigma(u), eps(v)) * ufl.dx
l = ufl.inner(b, v) * ufl.dx


## Boundary conditions

Left: 

$$ u_1 = 0$$

Bottom: 

$$ u_2 = 0$$

Right: 

$$ u_1 = \varepsilon_0 L\\  
   u_2 =0
$$

In [82]:
# Get degrees of freedom (DOFs)
left_dofs = fem.locate_dofs_topological(V.sub(0), 1, left_facets)
right_dofs = fem.locate_dofs_topological(V, 1, right_facets)
bottom_dofs = fem.locate_dofs_topological(V.sub(1), 1, bottom_facets)

# Constrain DOFs
bc_left = fem.dirichletbc(PETSc.ScalarType(0), dofs=left_dofs, V=V.sub(0))
bc_right = fem.dirichletbc(
    np.array((EPS0 * L, 0), dtype=PETSc.ScalarType), dofs=right_dofs, V=V
)
bc_bottom = fem.dirichletbc(PETSc.ScalarType(0), dofs=bottom_dofs, V=V.sub(1))


## Assemble and solve

In [83]:
problem = fem.petsc.LinearProblem(
    a,
    l,
    bcs=[bc_left, bc_right, bc_bottom],
    petsc_options={"ksp_type": "preonly", "pc_type": "lu"},
)
uh = problem.solve()


## Postprocessing

In [110]:
# Set backend for plots
pyvista.set_jupyter_backend("client")
p = pyvista.Plotter(shape=(1, 3))

# Convert mesh to VTK for plotting with pyvista
topology, cell_types, x = plot.create_vtk_mesh(V)
grid = pyvista.UnstructuredGrid(topology, cell_types, x)

# Displacement data ('u' is a 3D vector for warping the mesh)
u = np.zeros((x.shape[0], 3))
u[:, : len(uh)] = uh.x.array.reshape((x.shape[0], len(uh)))
grid["u"] = u
grid["u_x"] = u[:,0]
grid["u_y"] = u[:,1]

# Stress data (evaluated at the interpoltion points, i.e. this is constant in each  
# element for linear elements)
T = fem.TensorFunctionSpace(domain,("DG", 0))
stress_expr = fem.Expression(sigma(uh), T.element.interpolation_points())
stress = fem.Function(T)
stress.interpolate(stress_expr)
s = stress.x.array.reshape((len(cell_types), 4))
grid["s"] = s

# Stress in x
p.subplot(0, 0)
p.add_mesh(grid.warp_by_vector("u", factor=1.0), component=0, show_edges=True, scalars="s")
p.view_xy()

# Displacement in x
p.subplot(0, 1)
p.add_mesh(grid.warp_by_vector("u", factor=1.0), show_edges=True, scalars="u_x", cmap="inferno")
p.view_xy()

# Displacement in x
p.subplot(0, 2)
p.add_mesh(grid.warp_by_vector("u", factor=1.0), show_edges=True, scalars="u_y", cmap="inferno_r")
p.view_xy()

# Show plot
p.show()


Widget(value="<iframe src='http://localhost:55051/index.html?ui=P_0x16b925660_10&reconnect=auto' style='width:…